# Experiment Setup

In [ ]:
# % TODO: State all conditions and experiment setup - how many randomly generated victims?
# % Initialise an large area with multiple hotspots
# % Sum all the probabilities of generating victims in said hotspots
# % Generate hotspots

# % Cluster, task and search 
# % Run 100 trials and compare 
#     % Minimum time capture
#     % Speed of capture (average time steps),
#     % First and last capture
#     % Path coverage

import numpy as np
import folium
from scipy.stats import multivariate_normal

boundary_pts = [
    (1.34554, 103.95949),
    (1.34552, 103.96812),
    (1.33954, 103.96831),
    (1.33964, 103.95952),
]

n_hotspots = 10
hotspots = [
    (1.3408531, 103.9611108),
    (1.3409068, 103.9621181),
    (1.3402594, 103.9621944),
    (1.3447965, 103.9637608),
    (1.3418683, 103.9650483),
    (1.3411711, 103.9653916),
    (1.3443996, 103.9603276),
    (1.3443996, 103.9648981),
    (1.3433592, 103.9626021),
    (1.3450968, 103.9611108)
]

hotspots_np = np.array(hotspots)


##### Generate victims around hotspots using Gaussian distribution #####
n_victims = 15
victims = []

# Get grid and pdf for each coordinate
grid_x, grid_y = np.mgrid[1.33964:1.34552:100j, 103.95949:103.96831:100j]
pos = np.dstack((grid_x, grid_y))
combined_pdf = np.zeros(pos.shape[:2])
covariance_matrix = [[0.00000001, 0], [0, 0.00000001]]

for hotspot in hotspots:
    rv = multivariate_normal(hotspot, covariance_matrix)
    combined_pdf += rv.pdf(pos)

# Normalize the combined PDF to avoid NaNs
combined_pdf /= np.sum(combined_pdf)

boundary_pts_np = np.array(boundary_pts)

# combined_pdf.shape # is the probability associated with (100, 100) 
victim_indices = np.random.choice(np.arange(combined_pdf.size), p=combined_pdf.ravel(), size=n_victims)
victim_coords = np.column_stack(np.unravel_index(victim_indices, combined_pdf.shape))
victims = np.column_stack((grid_x[victim_coords[:, 0], victim_coords[:, 1]], grid_y[victim_coords[:, 0], victim_coords[:, 1]]))


##### VISUALISATION: Plot the map with victims and hotspots #####
map_center = [np.mean(boundary_pts_np[:, 0]), np.mean(boundary_pts_np[:, 1])]
m = folium.Map(location=map_center, zoom_start=15)

# Add boundary points to the map
# for lat, lon in boundary_pts:
#     folium.Marker(location=[lat, lon], popup='Boundary Point', icon=folium.Icon(color='blue')).add_to(m)

boundary_pts_polyline = boundary_pts 
boundary_pts_polyline.append(boundary_pts[0])
line=folium.PolyLine(locations=boundary_pts_polyline,weight=3,color="#FF0000").add_to(m)

# Victim
for lat, lon in victims:
    folium.Marker(location=[lat, lon], popup='Victim', icon=folium.Icon(color='green')).add_to(m)

In [ ]:
display(m)

# Search Setup

In [ ]:
from run_clustering import run_clustering

def clustering_step(hotspots, threshold):
    clusters = run_clustering(hotspots, threshold=threshold)
    clusters = list(clusters.values())
    clusters = sorted(clusters, key=lambda x:len(x[1]), reverse=False)

    for i in range(len(clusters)):
        if len(clusters[i][1]) == 1:
            cluster_coord = clusters[i][0]
            folium.Marker(location=[cluster_coord[0], cluster_coord[1]], popup=f'Cluster{i}+Point', icon=folium.Icon(color='black')).add_to(m)
        else:
            cluster_coord = clusters[i][0]
            folium.Marker(location=[cluster_coord[0], cluster_coord[1]], popup=f'Cluster{i}', icon=folium.Icon(color='black')).add_to(m)
            for j in range(len(clusters[i][1])):
                hotspot_coord = clusters[i][1][j]
                folium.Marker(location=[hotspot_coord[0], hotspot_coord[1]], popup=f'hotspot_{i}', icon=folium.Icon(color='red')).add_to(m)
    return clusters


In [ ]:
# Constants
N_RINGS_CLUSTER = 50
DEFAULT_RESOLUTION = 14
MAX_NUMBER_STEPS = 3000  # Assuming a maximum number of steps to take
START_TUPLE = np.mean(boundary_pts, axis=0)

# Initialize variables
working_c = clustering_step(hotspots=hotspots, threshold=0.1)

In [ ]:
folium.Marker(location=[START_TUPLE[0], START_TUPLE[1]], popup=f'START', icon=folium.Icon(color='purple')).add_to(m)

In [ ]:
display(m)

In [ ]:
from typing import Tuple
import h3 

DEFAULT_RESOLUTION = 14
START_TUPLE = np.mean(boundary_pts, axis=0)

victim_hexagons = [h3.geo_to_h3(victim[0], victim[1], DEFAULT_RESOLUTION) for victim in victims]
print(victim_hexagons)

def check_if_victim_is_detected(location: Tuple):
    g = h3.geo_to_h3(location[0], location[1], DEFAULT_RESOLUTION)
    if g in victim_hexagons:
        return victim_hexagons.index(g)
    else:
        return None

In [ ]:
from clusterfinder.maplib import LatLon
from pathfinder.pathfinder import BayesianHexSearch, OutwardSpiralPathFinder, init_empty_prob_map, update_prob_map_w_hotspots, update_probability_map
from typing import List, Tuple


#TODO:
# Increase the number of victims
# Tune all the other parameters
# N_RING_CLUSTER has an exponential computing time
# max_number_steps check if it makes sense logically for length of flight
PROBABILITY_DECAY=0.95

n_drones = 4  # Number of drones
available_drones = [i for i in range(n_drones)]
step_count = [0 for _ in range(4)]
drone_current_pos = [START_TUPLE for _ in range(4)]
next_round_drones = []

detected_history = {}

print(len(working_c))
print(len(available_drones))
# Main loop
while len(working_c) > 0:
    if len(working_c) == 0:
        break
    else:
        c = working_c.pop() # [0] is location of cluster, [1] is list of hotspots

    drone = available_drones.index(step_count.index(min(step_count))) # Take the drone with minimum number of steps

    # Explore using drone
    # Go to cluster center
    path_to_cluster_centre = h3.h3_line(h3.geo_to_h3(drone_current_pos[drone][0], drone_current_pos[drone][1], DEFAULT_RESOLUTION),
                                        h3.geo_to_h3(c[0][0], c[0][1], DEFAULT_RESOLUTION))
    step_count[drone] += len(path_to_cluster_centre) - 1 # not inclusive of current cell
    drone_current_pos[drone] = (c[0][0], c[0][1])

    # Initialize probability map
    target_pos = c[0]
    target_pos_latlon = LatLon(target_pos[0], target_pos[1])
    prob_map = init_empty_prob_map(target_pos_latlon, N_RINGS_CLUSTER)
    prob_map = update_prob_map_w_hotspots(probability_map=prob_map, hotspots=c[1])
    # hotspots = cluster[1]

    # # Initialize pathfinder
    pathfinder = BayesianHexSearch(DEFAULT_RESOLUTION, center=target_pos)

    path = []
    for i in range(MAX_NUMBER_STEPS):
        # print(i)
        drone_current_pos[drone] = pathfinder.find_next_step(drone_current_pos[drone], prob_map)
        prob_map = update_probability_map(prob_map, drone_current_pos[drone], PROBABILITY_DECAY)
        if i%10==0: path.append(drone_current_pos[drone])
        step_count[drone] += 1

        detected = check_if_victim_is_detected(drone_current_pos[drone])
        if detected:
            # Record the time step when the victim is detected
            if detected not in detected_history.keys():
                detected_history[detected] = step_count[drone]
                print(f"Drone {drone} detected a victim at step {step_count[drone]}")
        
        line=folium.PolyLine(locations=path,weight=1)
        m.add_child(line)
    # break
        # # Mark the drone as available again
        # available_drones.append(drone)
    # available_drones = next_round_drones.copy()

# Print the total number of victims found and the step count for each victim detection
# victims_found = sum([check_if_victim_is_detected(current_tuple[drone]) for drone in range(n_drones)])
# print(f"Total victims found: {victims_found}")
# print(f"Step counts: {step_count}")


In [ ]:
print(f'Number of detected:{len(detected_history)}/{n_victims}')
print(f'Steps{step_count}')
print(f'Average detected time of humans:{sum(detected_history.values())/len(detected_history)}')

In [ ]:
display(m)

In [ ]:
# Code the default policy for comparison

boundary_pts = [
    (1.34554, 103.95949),
    (1.34552, 103.96812),
    (1.33954, 103.96831),
    (1.33964, 103.95952),
]

In [ ]:
print(victim_hexagons)

In [ ]:
def check_if_victim_is_detected_h3(h3_cell: str):
    if h3_cell in victim_hexagons:
        return victim_hexagons.index(h3_cell)
    else:
        return None

In [ ]:
# Get the hexagons that cover the polygon area defined by boundary_pts
polygon = {
    'type': 'Polygon',
    'coordinates': [[
        [lng, lat] for lat, lng in boundary_pts
    ]]
}


hexagons = list(h3.polyfill(polygon, DEFAULT_RESOLUTION, geo_json_conformant=True))

# Convert hexagons to their lat/lng center points
hex_centers = {h: h3.h3_to_geo(h) for h in hexagons}

# Sort the hexagons by their lat/lng to create a zigzag pattern
sorted_hexagons = sorted(hexagons, key=lambda h: (hex_centers[h][0], hex_centers[h][1]))

# Create the zigzag pattern
def create_zigzag_path(hexagons, hex_centers):
    rows = {}
    for hex in hexagons:
        lat = round(hex_centers[hex][0], 6)
        if lat not in rows:
            rows[lat] = []
        rows[lat].append(hex)
    
    zigzag_path = []
    toggle = False
    for lat in sorted(rows.keys()):
        row = rows[lat]
        if toggle:
            zigzag_path.extend(reversed(row))
        else:
            zigzag_path.extend(row)
        toggle = not toggle

    return zigzag_path

zigzag_path = create_zigzag_path(sorted_hexagons, hex_centers)


zigzag_ = []
detected_history_base = {}
for i in range(len(zigzag_path)):
    if i%1000==0: zigzag_.append(h3.h3_to_geo(zigzag_path[i]))

    detected = check_if_victim_is_detected_h3(zigzag_path[i])
    if detected:
        if detected not in detected_history_base.keys():
            detected_history_base[detected] = i

line=folium.PolyLine(locations=zigzag_,weight=3)
m.add_child(line)

detected_divided = [step%(len(zigzag_path)/n_drones) for step in detected_history_base.values()]
print(detected_history_base)
print(detected_divided)

print(f'Length:{len(zigzag_path)}')
print(f'Average number of steps:{len(zigzag_path)/n_drones}')
print(f'Number found{len(detected_history_base)}/{n_victims}')
print(f'Average number of steps found {sum(detected_divided)/len(detected_divided)}')

In [ ]:
display(m)